# Querying the Guide to Pharmacology Database

---
This notebook initializes and works with the Guide to Pharmacology Database using Neo4j and Cypher.


## Setup and Data Loading

First, let's review the setup and ensure all necessary dependencies are installed.

### Installing Required Libraries

This cell installs the `rdflib-neo4j` and `openpyxl` libraries, which are necessary for RDF graph handling and working with Excel files, respectively.

In [ ]:
%pip install -q neo4j openpyxl

In [2]:
from pathlib import Path

# Define configuration constants
INTERIM_DIR = Path("../data/interim")  # Standardize path using pathlib

### Setting up Database Connection

Here, we configure the connection to the Neo4j database by using environment variables for secure access. Ensure the required environment variables (`NEO4J_LCL_URI`, `NEO4J_USERNAME`, and `NEO4J_LCL_PASSWORD`) are set before proceeding.

In [3]:
import os

NEO_DB_URI = os.getenv('NEO4J_URI')
NEO_DB_USERNAME = os.getenv('NEO4J_USERNAME')
NEO_DB_PWD = os.getenv('NEO4J_PASSWORD')


In [ ]:
print(f'NEO_DB_URI: {NEO_DB_URI}')

## Querying the Guide to Pharmacology Data

This section demonstrates how to query the database to explore the Guide to Pharmacology (GtoP) dataset using Cypher.


### Initializing Neo4j Driver

This cell initializes the Neo4j `GraphDatabase` driver for establishing a connection to the database. Ensure the credentials and URI are correctly configured for successful interaction.


In [5]:
from neo4j import GraphDatabase

# Initialize Neo4j driver
driver = GraphDatabase.driver(
    NEO_DB_URI,
    auth=(NEO_DB_USERNAME, NEO_DB_PWD)
)

#### Setup Reused Functions

This section introduces reusable helper functions to streamline querying and interacting with the Neo4j database.


In [6]:
def run_cypher_query(query):
    with driver.session(database="neo4j") as session:
        result = session.run(query)
        # Fetch all results and convert them into a list of dictionaries
        return [record.data() for record in result]

### Querying for a list of drug compounds

In [7]:
cypher_query = """
MATCH (l:Resource)
WHERE l.ligandName IN ['RGFP966', 'belinostat', 'ricolinostat', 'nexturastat A', 'pyroxamide', 'entinostat', 'resminostat', 'panobinostat', 'R306465', 'M 344', 'AR-42']
MATCH path = (l)-[:hasLigand]-(i)-[:hasTarget]-(t)-[:hasTargetFamily]-(tf)
    OPTIONAL MATCH (i)-[:hasAction]-(a)
    OPTIONAL MATCH (i)-[:hasAffinity]-(af)
    OPTIONAL MATCH (i)-[:hasReference]-(r)
    OPTIONAL MATCH (l)-[:xref]-(xr)
RETURN path, a, af, r
"""

In [ ]:
from pprint import pprint

results = run_cypher_query(cypher_query)

pprint(results)

In [9]:
export_query = f"""
CALL apoc.export.graphml.query(
    'MATCH (l:Resource)
     WHERE l.ligandName IN ["RGFP966", "belinostat", "ricolinostat", "nexturastat A", "pyroxamide", "entinostat", "resminostat", "panobinostat", "R306465", "M 344", "AR-42"]
     MATCH path = (l)-[:hasLigand]-(i)-[:hasTarget]-(t)-[:hasTargetFamily]-(tf)
     OPTIONAL MATCH (i)-[:hasAction]-(a)
     OPTIONAL MATCH (i)-[:hasAffinity]-(af)
     OPTIONAL MATCH (i)-[:hasReference]-(r)
     OPTIONAL MATCH (l)-[:xref]-(xr)
     RETURN path, a, af, r',
    null,
    {{stream: true, useTypes: true}}
)
YIELD data
RETURN data
"""

with driver.session() as session:
    result = session.run(export_query)
    graphml_data = result.single()['data']


In [10]:
# Write the GraphML data to a file
with open(f"{INTERIM_DIR}/gtp_aurk_inhibitors.graphml", 'w') as f:
    f.write(graphml_data)

### Querying for Lists of Compounds - Tabular Format

This section begins querying the database for specific information, such as retrieving lists of compounds using Cypher.


In [11]:
cypher_query_hdac = """
// HDAC Inhibitors
MATCH (l:Resource)
WHERE l.ligandName IN ['RGFP966', 'belinostat', 'ricolinostat', 'nexturastat A', 'pyroxamide', '5-nitroso-8-quinolinol', 'entinostat', 'resminostat', 'panobinostat', 'R306465', 'M 344', 'AR-42']
MATCH path = (l)-[:hasLigand]-(i)-[:hasTarget]-(t)-[:hasTargetFamily]-(tf)
    OPTIONAL MATCH (i)-[:hasAction]-(a)
    OPTIONAL MATCH (i)-[:hasAffinity]-(af)
    OPTIONAL MATCH (i)-[:hasReference]-(r)
    OPTIONAL MATCH (l)-[:xref]-(xr)
    OPTIONAL MATCH (af)-[:hasUnits]-(afu)
RETURN 
    l.label as Compound,
    l.approved as FDA_Approved,
    t.label as Target,
    tf.label as Target_Family,
    a.uri as Action,
    af.hasMedianValue as Affinity_Median,
    af.hasLowValue as Affinity_Low,
    af.hasHighValue as Affinity_High,
    afu.uri as Affinity_Units,
    r.uri as Reference,
    l.comment as Description,
    l.inChIKey as InChIKey,
    l.canonicalSMILES as SMILES,
    xr.uri as CHEMBL
ORDER BY l.ligandName, t.nomenclature
"""

cypher_query_cdk = """
// CDK Inhibitors
MATCH (l:Resource)
WHERE l.ligandName IN ['indisulam', 'dinaciclib', 'flavopiridol', 'Bms-265246', 'AT7519', 'PHA-793887', 'riviciclib', 'R-547', '7-hydroxystaurosporine', 'zotiraciclib', 'Sns-032', 'AZD5438']
MATCH path = (l)-[:hasLigand]-(i)-[:hasTarget]-(t)-[:hasTargetFamily]-(tf)
    OPTIONAL MATCH (i)-[:hasAction]-(a)
    OPTIONAL MATCH (i)-[:hasAffinity]-(af)
    OPTIONAL MATCH (i)-[:hasReference]-(r)
    OPTIONAL MATCH (l)-[:xref]-(xr)
    OPTIONAL MATCH (af)-[:hasUnits]-(afu)
RETURN 
    l.label as Compound,
    l.approved as FDA_Approved,
    t.label as Target,
    tf.label as Target_Family,
    a.uri as Action,
    af.hasMedianValue as Affinity_Median,
    af.hasLowValue as Affinity_Low,
    af.hasHighValue as Affinity_High,
    afu.uri as Affinity_Units,
    r.uri as Reference,
    l.comment as Description,
    l.inChIKey as InChIKey,
    l.canonicalSMILES as SMILES,
    xr.uri as CHEMBL
ORDER BY l.ligandName, t.nomenclature
"""

cypher_query_aurk = """
// AURK Inhibitors
MATCH (l:Resource)
WHERE l.ligandName IN ['AZD1152-HQPA', 'tozasertib', 'SNS-314 mesylate', 'AMG 900', 'ZM-447439', 'ENMD-2076', 'CYC116', 'alisertib', 'PF-03814735', 'CCT129202', 'CCT137690']
MATCH path = (l)-[:hasLigand]-(i)-[:hasTarget]-(t)-[:hasTargetFamily]-(tf)
    OPTIONAL MATCH (i)-[:hasAction]-(a)
    OPTIONAL MATCH (i)-[:hasAffinity]-(af)
    OPTIONAL MATCH (i)-[:hasReference]-(r)
    OPTIONAL MATCH (l)-[:xref]-(xr)
    OPTIONAL MATCH (af)-[:hasUnits]-(afu)
RETURN 
    l.label as Compound,
    l.approved as FDA_Approved,
    t.label as Target,
    tf.label as Target_Family,
    a.uri as Action,
    af.hasMedianValue as Affinity_Median,
    af.hasLowValue as Affinity_Low,
    af.hasHighValue as Affinity_High,
    afu.uri as Affinity_Units,
    r.uri as Reference,
    l.comment as Description,
    l.inChIKey as InChIKey,
    l.canonicalSMILES as SMILES,
    xr.uri as CHEMBL
ORDER BY l.ligandName, t.nomenclature
"""

In [12]:
import pandas as pd

# Execute the query and display the results
results_hdac = run_cypher_query(cypher_query_hdac)
results_cdk = run_cypher_query(cypher_query_cdk)
results_aurk = run_cypher_query(cypher_query_aurk)

### Creating DataFrame for HDAC Data

Converts the results of the HDAC inhibitors Cypher query into a Pandas DataFrame for structured data representation and further processing.


In [ ]:
df_hdac = pd.DataFrame(results_hdac)
df_hdac.head()

### Creating DataFrame for CDK Data

Converts the results of the CDK inhibitors Cypher query into a Pandas DataFrame for structured data representation and further processing.


In [ ]:
df_cdk = pd.DataFrame(results_cdk)
df_cdk.head()

### Creating DataFrame for AURK Data

Converts the results of the AURK inhibitors Cypher query into a Pandas DataFrame for structured data representation and further processing.


In [ ]:
df_aurk = pd.DataFrame(results_aurk)
df_aurk.head()

### Exporting DataFrames to Excel Files

Exports the processed DataFrames for HDAC, CDK, and AURK inhibitors into separate Excel files. This allows external analysis or sharing of the data.


In [16]:
# export each DataFrame to a xlsx file
df_hdac.to_excel(f"{INTERIM_DIR}/gtp_hdac_inhibitors_data.xlsx", index=False)
df_cdk.to_excel(f"{INTERIM_DIR}/gtp_cdk_inhibitors_data.xlsx", index=False)
df_aurk.to_excel(f"{INTERIM_DIR}/gtp_aurk_inhibitors_data.xlsx", index=False)

In [17]:
driver.close()

## Citation

**BibTeX:**

```
@article{10.1093/nar/gkad944,
    author = {Harding, Simon D and Armstrong, Jane F and Faccenda, Elena and Southan, Christopher and Alexander, Stephen P H and Davenport, Anthony P and Spedding, Michael and Davies, Jamie A},
    title = "{The IUPHAR/BPS Guide to PHARMACOLOGY in 2024}",
    journal = {Nucleic Acids Research},
    volume = {52},
    number = {D1},
    pages = {D1438-D1449},
    year = {2023},
    month = {10},
    abstract = "{The IUPHAR/BPS Guide to PHARMACOLOGY (GtoPdb; https://www.guidetopharmacology.org) is an open-access, expert-curated, online database that provides succinct overviews and key references for pharmacological targets and their recommended experimental ligands. It includes over 3039 protein targets and 12 163 ligand molecules, including approved drugs, small molecules, peptides and antibodies. Here, we report recent developments to the resource and describe expansion in content over the six database releases made during the last two years. The database update section of this paper focuses on two areas relating to important global health challenges. The first, SARS-CoV-2 COVID-19, remains a major concern and we describe our efforts to expand the database to include a new family of coronavirus proteins. The second area is antimicrobial resistance, for which we have extended our coverage of antibacterials in partnership with AntibioticDB, a collaboration that has continued through support from GARDP. We discuss other areas of curation and also focus on our external links to resources such as PubChem that bring important synergies to the resources.}",
    issn = {0305-1048},
    doi = {10.1093/nar/gkad944},
    url = {https://doi.org/10.1093/nar/gkad944},
    eprint = {https://academic.oup.com/nar/article-pdf/52/D1/D1438/55039511/gkad944.pdf},
}
```